In [2]:
# !pip install fosforml

In [1]:
# pip show fosforml

## Sklearn Models Registration

### session creation

In [1]:
from fosforml.model_manager.snowflakesession import get_session, get_connection_params
session = get_session()
connection_params = get_connection_params()

### modeling

#### Multiclass classification

In [12]:
# from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble._forest import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd

m_dataset = load_iris() 
feature_names = m_dataset.feature_names
u_features = [col.replace(" ","_").replace("(","").replace(")","") for col in feature_names]
m_df = pd.DataFrame(m_dataset.data,columns=u_features)
m_df["Target"] = m_dataset.target


x_train,x_test,y_train,y_test = train_test_split(m_df.iloc[:,:-1],m_df['Target'])
model = DecisionTreeClassifier()
model.fit(x_train,y_train)

y_pred = pd.DataFrame(model.predict(x_test),columns=["Predicted"])

In [13]:
df = pd.read_csv("HR_CHURN.csv").drop('Unnamed: 0',axis=1)

In [14]:
x_train,x_test,y_train,y_test = train_test_split(df.drop('SALARY',axis=1),df['SALARY'])

In [16]:
x_train.head()

,SATISFACTION_LEVEL,LAST_EVALUATION,NUMBER_PROJECT,AVERAGE_MONTLY_HOURS,TIME_SPEND_COMPANY,WORK_ACCIDENT,LEFT,PROMOTION_LAST_5YEARS,DEPARTMENT
12197,0.10,0.91,6,287,4,0,1,0,technical
7437,0.98,0.80,3,166,2,0,0,0,hr
3954,0.70,0.84,2,114,4,0,0,0,IT
7516,0.69,0.97,4,264,3,0,0,0,technical
13220,0.81,0.55,4,217,8,0,0,0,accounting


In [15]:
model = RandomForestClassifier()
model.fit(x_train,y_train)

ValueError: could not convert string to float: 'technical'

In [3]:
model

DecisionTreeClassifier()

In [5]:
# data = decision_tree(model,u_features)

In [6]:
from fosforml import register_model
register_model(
  model_obj=model,
  session=session,
  x_train=x_train,
  y_train=y_train,
  x_test=x_test,
  y_test=y_test,
  y_pred=y_pred,
  dataset_name="RandomMulticlassMOdelForTesting",
  dataset_source="Snowflake",
  name="RandomMultieefefefclassMOdelForTesting16July",
  description="This is a test sklearn model",
  flavour="sklearn",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.classification._register_confusion_matrix_computer.<locals>.ConfusionMatrixComputer'>. Proceeding without creating optional arguments
Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.metrics_utils.register_accumulator_udtf.<locals>.Accumulator'>. Proceeding without creating optional arguments


Calculating build time metrics

Progress: ██████████████                                                         20.0%
Calculating build time metrics

Progress: ████████████████████████████                                           40.0%


Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.classification._register_multilabel_confusion_matrix_computer.<locals>.MultilabelConfusionMatrixComputer'>. Proceeding without creating optional arguments
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn<1.4'. Your UDF might not work when the package version is different between the server and your local environment.
DataFrame.flatten() is deprecated since 0.7.0. Use `DataFrame.join_table_function()` instead.
Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.classification._register_multilabel_confusion_matrix_computer.<locals>.MultilabelConfusionMatrixComputer'>. Proceeding without creating optional arguments
The version of package 'scikit-learn' in the local environment is 1.

Calculating build time metrics

Progress: ██████████████████████████████████████████                             60.0%
Calculating build time metrics

Progress: ████████████████████████████████████████████████████████               80.0%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


"Model 'MODEL_8D3FB8A6_A886_4236_9B35_73EB1303C4BF_FDC_RANDOMMULTIEEFEFEFCLASSMODELFORTESTING16JULY' registered successfully."